# DATA620: Project 3 - Gender Identification

## Homework Team 3: David Simbandumwe, Eric Lehmphul and Lidiia Tronina

Using any of the three classifiers described in chapter 6 of Natural Language Processing with Python, and any features you can think of, build the best name gender classifier you can. Begin by splitting the Names Corpus into three subsets: 500 words for the test set, 500 words for the dev-test set, and the remaining 6900 words for the training set. Then, starting with the example name gender classifier, make incremental improvements. Use the dev-test set to check your progress. Once you are satisfied with your classifier, check its final performance on the test set. How does the performance on the test set compare to the performance on the dev-test set? Is this what you'd expect?

<br>
We chose to use the NaiveBayesClassifier machine learning method to categorize the names Corpus. 

### Load Required Packages

In [ ]:
import nltk
from nltk.corpus import names
from nltk.classify import apply_features
from nltk.probability import FreqDist
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from sklearn.model_selection import KFold
from statistics import mean
import plotly.express as px


<br>

### Model Base Class
This is a Generic class with base model and boiler plate analysis code. It is used to simplify the analysis for each subsequent model.

In [ ]:

# generic class with base model
class GenderAnalysis():
    
    # set corpus and training datasets
    def __init__(self):
       
        random.seed(3210)
        self._classifier = None
        
        # Import and Lable Names
        self._names_lst = ([(name, 'male') for name in names.words('male.txt')] +
                [(name, 'female') for name in names.words('female.txt')])
        random.shuffle(self._names_lst)

        # Split Corpus
        self._train_names = self._names_lst[1000:]
        self._devtest_names = self._names_lst[500:1000]
        self._test_names = self._names_lst[:500]
    
        # apply features training, devtest and test corpus
        self._train_set = apply_features(self.gender_features, self._train_names)
        self._devtest_set = apply_features(self.gender_features, self._devtest_names)
        self._test_set = apply_features(self.gender_features, self._test_names)          

 
    # generate features
    def gender_features(self, name):
        pass
    
    
    # get features dataframe 
    def get_names_features(self):
        # analyze data
        df = pd.DataFrame(self._names_lst, columns=['name','gender'])
        return df
    
    
    # get names that can be both male and female
    def ambiguous_names(self):
        # analyze data
        df = pd.DataFrame(self._names_lst, columns=['name','gender'])
        df2 = df.groupby(['name'])['name'].count().reset_index(name='count').sort_values(
            ['count'], ascending=False)
        df2 = df2[df2['count']>1]
        name_df = pd.merge(df,df2,how="inner", on=["name"])
        
        #name_df = name_df.drop('count', 1)
        name_df = name_df[['name']]
        name_df = name_df.groupby('name').first().reset_index()
        
        return name_df
    
    
    # train the model
    def train(self):
        self._classifier = nltk.NaiveBayesClassifier.train(self._train_set)
        
        
    # return the accuracy of the dev test
    def get_devtest_accuracy(self):     
        return nltk.classify.accuracy(self._classifier, self._devtest_set)
        

    # return the features with the biggest impact
    def show_inform_feature(self, n=20):
        self._classifier.show_most_informative_features(n)
        
    
    # return a dataframe with the errors
    def get_errors(self):
        errors = []
        for (name, tag) in self._devtest_names:
            guess = self._classifier.classify(self.gender_features(name))
            if guess != tag:
                errors.append((tag, guess, name))                
          
        errors_df = pd.DataFrame(errors, columns=['tag', 'guess', 'name'])
        return errors_df
    
    
    # print the model errors
    def print_Errors(self):
        for (tag, guess, name) in sorted(errors):
            print ('correct=%-8s guess=%-8s name=%-30s' % (tag, guess, name))

     
    # print the errors that are driven by Unisex names
    def get_ambiguous_errors(self):
        df1 = self.ambiguous_names()
        df2 = self.get_errors()
        error_df = pd.merge(df1,df2,how="inner", on=["name"])
        return error_df
    
    
    # determint the accuracy of the model using test data
    def get_test_accuracy(self):
        return nltk.classify.accuracy(self._classifier, self._test_set)
    
    # explore of error records
    def get_error_stats(self):
              
        df = self.get_errors()
        
        df['length'] = df['name'].str.len()
        df['last_letter'] = df['name'].str[-1]
        df['first_letter'] = df['name'].str[0]

        df['vowels'] = df['name'].str.replace(r'[^aeiou]', '', regex=True)
        df['vowel_last_letter'] = df['last_letter'].isin([*'aeiouy'])
        df['vowel_first_letter'] = df['first_letter'].isin([*'AEIOUY'])

        df['consonants'] = df['name'].str.replace(r'[aeiou]', '', regex=True)
        df['consonant_last_letter'] = df['last_letter'].isin([*'qwrtplkjhgfdszxcvbnm'])
        df['consonant_first_letter'] = df['first_letter'].isin([*'QWRTPLKJHGFDSZXCVBNM'])

        df['num_vowels'] = df['vowels'].str.len()
        df['num_consonants'] = df['consonants'].str.len()
        
        return df
    

<br>

### K-Fold Sub Class
The K-Fold sub class adds a method to the generic class that genereates test stats for the k-fold validation

In [ ]:
# generic class with base model
class GenderAnalysisKFold(GenderAnalysis):

       
    # get kfold
    def get_kfold(self, n=10):
        
        cross_val_names = self._names_lst[500:]

        kf = KFold(n_splits=n)
        variables = apply_features(self.gender_features, cross_val_names)
        
        
        test_accuracy = {}
        sum = 0
        i = 1
        for train, test in kf.split(variables):
            train_data = np.array(variables)[train]
            test_data = np.array(variables)[test]
            classifier = nltk.NaiveBayesClassifier.train(train_data)
            test_accuracy.update({'k_'+str(i):round(nltk.classify.accuracy(classifier, test_data),3)})           
            i += 1
        
        test_accuracy.update({'k_average':round(mean(test_accuracy.values()),3)})        
        return pd.DataFrame([test_accuracy])

<br>

### Model Sub-classes
Each Model class inherits from the GenderAnalusis base class. The key difference between each subclass is the implimention of the gender_features().

In [ ]:

# Uses the last letters as features 
class GenderAnalysis1(GenderAnalysisKFold):
    
    def gender_features(self, name):
        features = {}
        features["suffix1"] = name[-1].lower()
        return features



# Uses the last 2 letters as features 
class GenderAnalysis2(GenderAnalysisKFold):
    
    def gender_features(self, name):
        features = {}
        features["suffix1"] = name[-1].lower()
        features["suffixLast2"] = name[-2:].lower()

        return features

    
# Use the last 2 letter and the first letter as features
class GenderAnalysis3(GenderAnalysisKFold):
    
    def gender_features(self, name):
        features = {}
        features["firstLetter"] = name[0].lower()
        features["suffix1"] = name[-1].lower()
        features["suffixLast2"] = name[-2:].lower()

        return features
    

# Add the vowels as first and last charater and vowel count 
class GenderAnalysis4(GenderAnalysisKFold):
    
    
    def gender_features(self, name):
        features = {}
        features["length"] = len(name)
        features["firstLetter"] = name[0].lower()
        features["lastLetter"] = name[-1].lower()
        features["firstVowel"] = features['firstLetter'] in 'aeiouy'
        features["lastVowel"] = features['lastLetter'] in 'aeiouy'

        # Gets num of vowels
        count = 0
        vowel = set("aeiouAEIOU")

        for i in name:
            if i in vowel:
                count = count + 1

        features["numVowels"] = count
        features["suffixLast2"] = name[-2:].lower()
        features["suffixLast3"] = name[-3:].lower()

        return features

<br>

### Analysis of Names Corpus
The corpus includes 365 Unisex names that have both male and female records. 

In [ ]:
ga1 = GenderAnalysis1()
ga1.train()

# Use the venn2 function
df = ga1.get_names_features()
s1 = set(df[df['gender'] == 'male']['name'])
s2 = set(df[df['gender'] == 'female']['name'])
venn2([s1, s2], set_labels = ('male', 'female'))
plt.show()

This Venn diagram captures that overlap of unisex names between the male and female populations. Interestingly enough there are many more female names in the corpus than male names. 

<br>

### Further Exploration of Names Corpus


In [ ]:
# explore data further
df['length'] = df['name'].str.len()
df['last_letter'] = df['name'].str[-1]
df['first_letter'] = df['name'].str[0]

df['vowels'] = df['name'].str.replace(r'[^aeiouyAEIOUY]', '', regex=True)
df['vowel_last_letter'] = df['last_letter'].isin([*'aeiouy'])
df['vowel_first_letter'] = df['first_letter'].isin([*'AEIOUY'])

df['consonants'] = df['name'].str.replace(r'[aeiouyAEIOUY]', '', regex=True)
df['consonant_last_letter'] = df['last_letter'].isin([*'qwrtplkjhgfdszxcvbnm'])
df['consonant_first_letter'] = df['first_letter'].isin([*'QWRTPLKJHGFDSZXCVBNM'])

df['num_vowels'] = df['vowels'].str.len()
df['num_consonants'] = df['consonants'].str.len()

df.head()

In [ ]:
df.groupby('gender').mean()

Notable differences between groups in the corpus:
* Female names tend to be longer
* Female names are much more likely to contain a vowel as the last letter
* Female names are slightly more likely to have a vowel as a first letter
* Males are much more likely to have a consonant for a last letter
* Both groups are almost equally likely to have a consonant as a first letter
* Females have more vowels on average
* Males have more consonants on average

This information provides us with a strong baseline for features to use in our gender name classifier.

In [ ]:
print('Size Train Set',len(ga1._train_names))
print('Size Development Set',len(ga1._devtest_names))
print('Size Test Set',len(ga1._test_names))



<br>

### Model 1
This is a simple model the looks at the last character of a name to determine the associated gender. This model has a 80% accuracy if we use 3210 as the seed. We can also count the number of errors that are driven by unisex names. With Model 1 there are 27 errors that can be attributed to unisex names.

In [ ]:
ga1 = GenderAnalysis1()
ga1.train()
print('devtest accuracy:', ga1.get_devtest_accuracy())
print('ambiguous errors:',len(ga1.get_ambiguous_errors()))

<br>

The feature list below identifies the features that have the biggest impact on the model. 

In [ ]:
ga1.show_inform_feature(10)

<br>

The table below summarizes the default model performance for each last letter. The accuracy levels and the resulting number of errors are high for:
* n - all names that end with n are classified as male
* e - all names that end in e are classified female
* y - all names that end in y are classified female

In [ ]:
# calculate the number of names in devtest that end with each letter
n_df = pd.DataFrame(ga1._devtest_names, columns=['name','gender'])
n_df['last_letter'] = n_df['name'].str[-1]
n_df = n_df.groupby(['last_letter'])['last_letter'].count().reset_index(name='count').sort_values(['count'], ascending=False)

# calculate the errors
e_df = ga1.get_errors()
e_df['last_letter'] = e_df['name'].str[-1]
e_df = e_df.groupby(['last_letter'])['last_letter'].count().reset_index(name='error_count')

# merge the dataframes
n_df = pd.merge(n_df,e_df,how="inner", on=["last_letter"])
n_df['accuracy'] = round(1 - n_df['error_count'] / n_df['count'],3)

# calculate the model accuracy for each letter
n_df.sort_values(['error_count'], ascending=False)

 <br>

### Model 2

This model extends the simple model and uses the last letter and the last 2 letters as features for name categorization. This model has a 82.8% accuracy if we use 3210 as the seed. We can also count the number of errors that are driven by unisex names. With Model 2 there are 21 errors that can be attributed to unisex names.

In [ ]:
ga2 = GenderAnalysis2()
ga2.train()
print('devtest accuracy:', ga2.get_devtest_accuracy())
print('ambiguous errors:',len(ga2.get_ambiguous_errors()))

<br>

The feature list below identifies the features that have the biggest impact on the model. It should be noted that suffixLast2 features dominate the top features list. Indicating that using 2 of the last characters is more deterministic of gender. 

In [ ]:
ga2.show_inform_feature(10)

<br>

The errors in name gender categorization are summarized in the following data frame. It is difficult to identify patterns at the individual name level so we will run some sample stats

In [ ]:
e_df = ga2.get_errors()
e_df.head()

<br>

The error data still shows some discrepancy in the vowel as last letter and the constant as last letter.

In [ ]:
e_df = ga2.get_error_stats()
e_df.groupby('tag').mean()

<br>

### Model 3
This model extends the simple model and uses first letter, the last letter, the last 2 letters as features for name categorization.  This model has a 84.4% accuracy if we use 3210 as the seed. We can also count the number of errors that are driven by unisex names. With Model 3 there are 24 errors that can be attributed to unisex names.


In [ ]:
ga3 = GenderAnalysis3()
ga3.train()

print('devtest accuracy:', ga3.get_devtest_accuracy())
print('ambiguous errors:',len(ga3.get_ambiguous_errors()))

<br>

The feature list below identifies the features that have the biggest impact on the model. It should be noted that suffixLast2 features dominate the top features list and the first letter does not appear as a top 20 feature . Indicating that using 2 of the last characters is more deterministic of gender.

In [ ]:
ga3.show_inform_feature(10)

<br>

The errors in name gender categorization are summarized in the following data frame. It is difficult to identify patterns at the individual name level so we will run some sample stats

In [ ]:
e_df = ga3.get_errors()
e_df.head()

The error data analysis still shows two discrepancy in the vowel as last letter and the constant as last letter gender breakdown. 

In [ ]:
e_df = ga3.get_error_stats()
e_df.groupby('tag').mean()

<br>

### Model 4

This model extends the simple model and uses first letter, the last letter, the last 2 letters as features for name categorization. It also uses the length of the name and the vowel counts as features for the name. This model has a 84.2% accuracy if we use 3210 as the seed. We can also count the number of errors that are driven by unisex names. With Model 4 there are 22 errors that can be attributed to unisex names.

In [ ]:
ga4 = GenderAnalysis4()
ga4.train()


print('devtest accuracy:', ga4.get_devtest_accuracy())
print('ambiguous errors:',len(ga4.get_ambiguous_errors()))


<br>

The feature list below identifies the features that have the biggest impact on the model. It should be noted that suffixLast2 features dominate the top features list and the first letter or the vowel related features do not appear as a top 20 feature.

In [ ]:
ga4.show_inform_feature(10)

The errors in name gender categorization are summarized in the following data frame. It is difficult to identify patterns at the individual name level so we will run some sample stats

In [ ]:
e_df = ga4.get_errors()
e_df.head()

The error data analysis still shows two discrepancy in the vowel as last letter and the constant as last letter gender breakdown.

In [ ]:
e_df = ga3.get_error_stats()
e_df.groupby('tag').mean()

<br>

### Model Accuracy
The accuracy for name categorization estimates seems to platau at roughly 85%. There appears to be some room to improve based on the treatment of vowels and consonants as the last letter. 

In [ ]:
result_lst = []
result_lst.append(['ga1',ga1.get_devtest_accuracy(),len(ga1.get_errors()),len(ga1.get_ambiguous_errors()),ga1.get_test_accuracy()])
result_lst.append(['ga2',ga2.get_devtest_accuracy(),len(ga2.get_errors()),len(ga2.get_ambiguous_errors()),ga2.get_test_accuracy()])
result_lst.append(['ga3',ga3.get_devtest_accuracy(),len(ga3.get_errors()),len(ga3.get_ambiguous_errors()),ga3.get_test_accuracy()])
result_lst.append(['ga4',ga4.get_devtest_accuracy(),len(ga4.get_errors()),len(ga4.get_ambiguous_errors()),ga4.get_test_accuracy()])

#result_df = pd.DataFrame(result_lst)
result_df = pd.DataFrame(result_lst, columns=['id','dt_accuracy','errors','ambiguous_errors','test_accuracy'])
result_df

<br>

### K-Fold Cross Validation
Using the K-fold method for cross validation we can examine how widely the performance varies across different training sets. K-fold cross validation will help us Identify the most "stable" model on out of sample data.

- First k fold produced the highest accuracy for each model
- Sixth k fold produced the lowest accuracy for each model
- Most of Model 1's results are within 1.5% of mean accuracy with the largest being 2.4%
- Most of Model 2's results are within 2% of mean accuracy with the largest being 3.7%
- Most of Model 3's results are within 2% of mean accuracy with the largest being 4.3%
- Most of Model 4's results are within 1.25% of mean accuracy with the largest being 3.7%

In [ ]:
# list
lst = []

# model 1
dict = ga1.get_kfold().to_dict(orient='index')[0]
dict.update({'id':'ga1'})
lst.append(dict)

# model 2
dict = ga2.get_kfold().to_dict(orient='index')[0]
dict.update({'id':'ga2'})
lst.append(dict)

# model 3
dict = ga3.get_kfold().to_dict(orient='index')[0]
dict.update({'id':'ga3'})
lst.append(dict)

# model 4
dict = ga4.get_kfold().to_dict(orient='index')[0]
dict.update({'id':'ga4'})
lst.append(dict)

# k folds dataframe
df = pd.DataFrame(lst)

In [ ]:
columns = list(df.columns[pd.Series(df.columns).str.startswith('k_')])
columns.remove('k_average')
l_df = pd.melt(df, id_vars =['id'], value_vars = columns, var_name ='fold', value_name ='accuracy' )

fig = px.box(
    data_frame=l_df, 
    title='Model Validation',
    y='accuracy',
    color='id')

# Show your work
fig.show()

In [ ]:
# summary data
pd.merge(left=result_df,right=df,left_on='id',right_on='id')

From the analysis we can see that Model 3 and Model 4 both perform well on all accuracy metrics, compared to the other models. Model 4 appears to be the better of the two models on out of sample performance, as it has the highest testing data accuracy and best average k-fold cross validation accuracy. Model 4 also had a fairly consistent accuracy across all k fold training sets, indicating that we can be fairly confident that the score is accurate. However it should be noted that this number can fluctuate depending on the ramdom seed selected.


<br>

### Conclusion
The difficulties associated with getting better accuracy in the gender classification names is impacted by the prevalence of unisex names. With 365 unisex names in the corpus it is common to see 24% to 28% of the errors are generated by unisex names that appear in corpus as both male and female names. 